In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import pickle

In [ ]:
books = pd.read_csv("airflow/dags/dataset/Books.csv")
ratings = pd.read_csv("airflow/dags/dataset/Ratings.csv")
users = pd.read_csv("airflow/dags/dataset/Users.csv")


C:\Users\AZTURAEV\AppData\Local\Temp\ipykernel_19392\1982076807.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("../dataset/Books.csv")


In [8]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [9]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [10]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [11]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [12]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [13]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [14]:
print("book", books.shape)
print("ratings", ratings.shape)
print("users", users.shape)



book (271360, 8)
ratings (1149780, 3)
users (278858, 3)


In [15]:
books[books.isnull()["Book-Author"]]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
118033,0751352497,A+ Quiz Masters:01 Earth,NaN,1999,Dorling Kindersley,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...
187689,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...


In [16]:
# # books.iloc[118033]["Book-Author"] = "unknown1"
# books.loc[118033, "Book-Author"] = "unknown1"
# books.loc[187689, "Book-Author"] = "unknown2"

In [17]:
books_copy = books.copy()
# books_copy = books_copy.drop(["Image-URL-M", "Image-URL-S"], axis=1)

In [18]:
data = books_copy.merge(ratings, how = "left", on = "ISBN")
data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2.0,0.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8.0,5.0
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400.0,0.0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676.0,8.0
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385.0,0.0


In [19]:
data.shape

(1032345, 10)

In [20]:
data.isnull().sum()

ISBN                      0
Book-Title                0
Book-Author               2
Year-Of-Publication       0
Publisher                 2
Image-URL-S               0
Image-URL-M               0
Image-URL-L               4
User-ID                1209
Book-Rating            1209
dtype: int64

In [21]:
data.dropna(subset=["User-ID"], inplace=True)
data.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            4
User-ID                0
Book-Rating            0
dtype: int64

In [22]:
data.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            4
User-ID                0
Book-Rating            0
dtype: int64

In [23]:
data["User-ID"] = [int(i) for i in data["User-ID"]]

In [24]:
# tmp_count = data.groupby("Book-Title").size().reset_index(name='Count')
# tmp_avg = data.groupby("Book-Title").mean("Book-Rating").reset_index().drop("User-ID", axis=1).round(2)
# merged_df = tmp_count.merge(tmp_avg, on="Book-Title")

tmp_count = data.groupby("ISBN").size().reset_index(name='Count')
tmp_avg = data.groupby("ISBN").mean("Book-Rating").reset_index().drop("User-ID", axis=1).round(2)
merged_df = tmp_count.merge(tmp_avg, on="ISBN")

In [55]:
# Select unique book metadata (assuming book details are the same for each title)
book_metadata = data[["Book-Title", "Book-Author", "Year-Of-Publication", "Publisher", 
                      "ISBN", "Image-URL-L"]].drop_duplicates()

book_metadata = book_metadata.set_index("ISBN")

book_metadata.head()

,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
ISBN,,,,,
0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...
0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...
0771074670,Nights Below Station Street,David Adams Richards,1988,Emblem Editions,http://images.amazon.com/images/P/0771074670.0...


In [26]:
merged_df.head()

,ISBN,Count,Book-Rating
0,0000913154,1,8.0
1,0001010565,2,0.0
2,0001046438,1,9.0
3,0001046713,1,0.0
4,000104687X,1,6.0


In [27]:
high_rated_ones = merged_df[merged_df["Count"]>50].sort_values("Book-Rating", ascending=False)[:100]
low_rated_ones = merged_df[merged_df["Count"]>50].sort_values("Book-Rating", ascending=False)[-100:]

In [28]:
high_rated_ones.head()

,ISBN,Count,Book-Rating
236927,1844262553,54,7.96
83625,0439064864,170,6.61
83867,0439139597,194,6.54
83846,0439136350,197,6.47
125011,0590353403,168,6.36


In [29]:
low_rated_ones.head()

,ISBN,Count,Book-Rating
44709,0345402308,53,1.66
10392,0061099732,73,1.66
86895,0440223237,60,1.65
91766,0446360589,150,1.65
87404,0440237300,57,1.65


In [30]:
with open("../server/server_artifacts/high_rated.pkl", "wb") as f:
    pickle.dump(high_rated_ones, f)

with open("../server/server_artifacts/low_rated.pkl", "wb") as f:
    pickle.dump(low_rated_ones, f)


In [31]:
# MemoryError: Unable to allocate
# the min possible is 7   
# Current RAM is 32 GB

remove_counts_less_than = 3

user_counts = data.groupby("User-ID").size()
low_activity_users = user_counts[user_counts <= remove_counts_less_than].index
data = data[~data["User-ID"].isin(low_activity_users)]


book_counts = data.groupby("ISBN").size()
low_rated_books = book_counts[book_counts <= remove_counts_less_than].index
data = data[~data["ISBN"].isin(low_rated_books)]



In [32]:
ratings["User-ID"].nunique()

105283

In [33]:
data.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Image-URL-S            0
Image-URL-M            0
Image-URL-L            0
User-ID                0
Book-Rating            0
dtype: int64

# Recommendation system

In [54]:
data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5.0
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0.0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8.0
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0.0
5,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,67544,8.0


In [35]:
user_item_matrix = data.pivot_table(index="User-ID", columns="ISBN", values="Book-Rating").fillna(0)

In [36]:
user_item_matrix

ISBN,0002005018,0002005050,0002190915,0002237458,0002239213,0002240114,0002243954,0002243962,0002244098,0002251760,...,9727110800,9812327975,9871106173,9871106181,9871138016,9871138148,9879397274,B00009EF82,B0000C37EK,B0000E63CJ
User-ID,,,,,,,,,,,,,,,,,,,,,
8,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# Standardize the data (helps with cosine similarity)
scaler = StandardScaler()
user_item_matrix_scaled = scaler.fit_transform(user_item_matrix)

In [38]:
# Compute cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix_scaled)

In [39]:
# Convert similarity matrix to a DataFrame
user_sim_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)


In [40]:
# Recommendation function
def recommend_books(user_id, num_recommendations=5):
    if user_id not in user_sim_df.index:
        return "User not found"

    # Find most similar users
    similar_users = user_sim_df[user_id].sort_values(ascending=False).iloc[1:6]

    # Get books rated highly by similar users
    similar_users_books = data[data["User-ID"].isin(similar_users.index)]
    
    # Find books the target user hasn't rated yet
    user_books = set(data[data["User-ID"] == user_id]["Book-Title"])
    candidate_books = similar_users_books[~similar_users_books["Book-Title"].isin(user_books)]
    
    # Aggregate ratings and sort by highest average rating
    # book_recommendations = candidate_books.groupby("Book-Title")["Book-Rating"].mean().sort_values(ascending=False).head(num_recommendations)
    book_recommendations = candidate_books.groupby("ISBN")["Book-Rating"].mean().sort_values(ascending=False).head(num_recommendations)
    book_recommendations = book_recommendations.index.tolist()

    book_recommendations = book_metadata.loc[book_recommendations]

    return book_recommendations
    # return book_recommendations.index.tolist()



In [41]:
user_id = 8

In [42]:
recommend_books(user_id)

,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
ISBN,,,,,
0140098798,The Progress of Love (King Penguin),Alice Munro,1987,Penguin Books,http://images.amazon.com/images/P/0140098798.0...
0446310786,To Kill a Mockingbird,Harper Lee,1988,Little Brown &amp; Company,http://images.amazon.com/images/P/0446310786.0...
0440212561,Outlander,DIANA GABALDON,1992,Dell,http://images.amazon.com/images/P/0440212561.0...
0684874350,ANGELA'S ASHES,Frank McCourt,1996,Scribner,http://images.amazon.com/images/P/0684874350.0...
0330412302,Don't Let's Go to Dogs Tonight: An African Chi...,Alexandra Fuller,2003,Pan Macmillan,http://images.amazon.com/images/P/0330412302.0...


In [43]:
with open("../server/server_artifacts/user_sim_matrix.pkl", "wb") as f:
    pickle.dump(user_sim_df, f)

with open("../server/server_artifacts/book_ratings.pkl", "wb") as f:
    pickle.dump(data, f)

In [44]:
with open("../server/server_artifacts/book_metadata.pkl", "wb") as f:
    pickle.dump(book_metadata, f)